### Training AutoEncoder


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define paths
combined_folder = '/content/drive/MyDrive/skin_cancer_images/combined'
autoencoder_model_path = '/content/drive/MyDrive/skin_cancer/autoencoder.keras'
encoder_model_path = '/content/drive/MyDrive/skin_cancer/encoder.keras'

# Autoencoder Architecture
input_shape = (224, 224, 3)  # Image size

# Encoder
input_img = Input(shape=input_shape)
x = Conv2D(32, (3,3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2,2), padding='same')(x)
x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
x = MaxPooling2D((2,2), padding='same')(x)
x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
bottleneck = MaxPooling2D((2,2), padding='same', name="encoder_output")(x)  # Bottleneck Layer

# Decoder
x = Conv2D(128, (3,3), activation='relu', padding='same')(bottleneck)
x = UpSampling2D((2,2))(x)
x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
x = UpSampling2D((2,2))(x)
x = Conv2D(32, (3,3), activation='relu', padding='same')(x)
x = UpSampling2D((2,2))(x)
output_img = Conv2D(3, (3,3), activation='sigmoid', padding='same')(x)  # Reconstructed Image

# Build Autoencoder Model
autoencoder = Model(input_img, output_img)
autoencoder.compile(optimizer='adam', loss='mse')

# Extract Encoder Model
encoder = Model(input_img, bottleneck)

# Create ImageDataGenerator for loading images
datagen = ImageDataGenerator(rescale=1./255)
data_generator = datagen.flow_from_directory(
    combined_folder,
    target_size=(224, 224),
    batch_size=32,
    class_mode='input',  # Change from 'None' to 'input' for autoencoder
    shuffle=True
)

# Train Autoencoder
autoencoder.fit(
    data_generator,
    epochs=10,  # Adjust based on performance
    steps_per_epoch=len(data_generator)
)

# Save Models
autoencoder.save(autoencoder_model_path)
encoder.save(encoder_model_path)
print(f"Autoencoder saved at: {autoencoder_model_path}")
print(f"Encoder saved at: {encoder_model_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 793 images belonging to 2 classes.
Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 148s 5s/step - loss: 0.0217
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 124ms/step - loss: 0.0139
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 129ms/step - loss: 0.0042
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 137ms/step - loss: 0.0020
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 125ms/step - loss: 0.0015
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 125ms/step - loss: 0.0020
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 148ms/step - loss: 0.0016
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 129ms/step - loss: 0.0012
Epoch 9/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 125ms/step - loss: 0.0011
Epoch 10/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 140ms/step - loss: 9.6163e-04
Autoencoder saved at: /content/drive/MyDrive/skin_cancer/autoencoder.keras
Encoder saved at: /content/drive/MyDrive/skin_cancer/encoder.keras


### Extracting features from auto encoder

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Paths
combined_folder = '/content/drive/MyDrive/skin_cancer_images/combined'
encoder_path = '/content/drive/MyDrive/skin_cancer/encoder.keras'
output_csv_path = '/content/drive/MyDrive/skin_cancer/autoencoder_features_with_labels.csv'

# Load the trained encoder
try:
    encoder = load_model(encoder_path)
    print(f"Encoder model loaded successfully from {encoder_path}")
except Exception as e:
    print(f"Error loading encoder: {e}")
    raise

# Create Data Generator for the dataset
datagen = ImageDataGenerator(rescale=1.0/255.0)  # Normalize pixel values
data_generator = datagen.flow_from_directory(
    combined_folder,
    target_size=(224, 224),  # Ensure it matches training dimensions
    batch_size=32,
    class_mode='binary',  # Get labels
    shuffle=False
)

# Extract latent features
all_features = []
all_labels = []

for batch_idx, (batch, labels) in enumerate(data_generator):
    features = encoder.predict(batch, verbose=0)  # Get bottleneck features
    all_features.append(features)
    all_labels.append(labels)

    # Stop after processing all images
    if (batch_idx + 1) * data_generator.batch_size >= data_generator.n:
        break

# Convert to structured format
if all_features:
    all_features = np.concatenate(all_features, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)
    reshaped_features = all_features.reshape(all_features.shape[0], -1)  # Flatten

    # Create DataFrame
    df = pd.DataFrame(reshaped_features)
    df['label'] = all_labels.astype(int)  # Append labels

    # Save to CSV
    df.to_csv(output_csv_path, index=False)
    print(f"Features with labels saved to {output_csv_path}")
else:
    print("No features extracted!")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Encoder model loaded successfully from /content/drive/MyDrive/skin_cancer/encoder.keras
Found 793 images belonging to 2 classes.
Features with labels saved to /content/drive/MyDrive/skin_cancer/autoencoder_features_with_labels.csv


### Code for Running SVM, NN, and RF

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Path to extracted features CSV
csv_path = '/content/drive/MyDrive/skin_cancer/autoencoder_features_with_labels.csv'

# Load features and labels
df = pd.read_csv(csv_path)
X = df.drop(columns=['label']).values  # Features
y = df['label'].values  # Labels

# Split dataset into train and test (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Standardize the features for better performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize classifiers
svm_model = SVC(kernel='rbf', C=1.0, random_state=42)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
nn_model = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=500, random_state=42)

# Train models
svm_model.fit(X_train, y_train)
rf_model.fit(X_train, y_train)
nn_model.fit(X_train, y_train)

# Make predictions
y_pred_svm = svm_model.predict(X_test)
y_pred_rf = rf_model.predict(X_test)
y_pred_nn = nn_model.predict(X_test)

# Calculate accuracy
accuracy_svm = accuracy_score(y_test, y_pred_svm)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
accuracy_nn = accuracy_score(y_test, y_pred_nn)

# Print results
print(f"SVM Accuracy: {accuracy_svm:.4f}")
print(f"Random Forest Accuracy: {accuracy_rf:.4f}")
print(f"Neural Network Accuracy: {accuracy_nn:.4f}")


SVM Accuracy: 0.7296
Random Forest Accuracy: 0.7107
Neural Network Accuracy: 0.7358


### Improving autoencoder



*   20 epochs
*    More Neurons in Latent Space (256): Helps retain more useful features.
*   More layers added for better feature extraction.






In [ ]:
import tensorflow.keras.backend as K
K.clear_session()


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define paths
combined_folder = '/content/drive/MyDrive/skin_cancer_images/combined'
autoencoder_model_path = '/content/drive/MyDrive/skin_cancer/improved_autoencoder.keras'
encoder_model_path = '/content/drive/MyDrive/skin_cancer/improved_encoder.keras'

# Clear GPU memory before training
tf.keras.backend.clear_session()

# Autoencoder Architecture (Memory Optimized)
input_shape = (160, 160, 3)  # Reduce image size to save memory

# Encoder
input_img = Input(shape=input_shape)
x = Conv2D(32, (3,3), activation='relu', padding='same')(input_img)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2), padding='same')(x)

x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2), padding='same')(x)

x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2), padding='same')(x)

bottleneck = Conv2D(256, (3,3), activation='relu', padding='same', name="encoder_output")(x)

# Decoder
x = Conv2D(128, (3,3), activation='relu', padding='same')(bottleneck)
x = BatchNormalization()(x)
x = UpSampling2D((2,2))(x)

x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = UpSampling2D((2,2))(x)

x = Conv2D(32, (3,3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = UpSampling2D((2,2))(x)

output_img = Conv2D(3, (3,3), activation='sigmoid', padding='same')(x)

# Build Autoencoder Model
autoencoder = Model(input_img, output_img)
autoencoder.compile(optimizer='adam', loss='mse')

# Extract Encoder Model
encoder = Model(input_img, bottleneck)

# Create ImageDataGenerator for loading images
datagen = ImageDataGenerator(rescale=1./255)
data_generator = datagen.flow_from_directory(
    combined_folder,
    target_size=(160, 160),  # Reduced image size
    batch_size=8,  # Reduced batch size
    class_mode='input',
    shuffle=True
)

# Train Autoencoder
autoencoder.fit(
    data_generator,
    epochs=15,  # Reduce epochs to save resources
    steps_per_epoch=200  # Limit steps per epoch to avoid OOM
)

# Save Models
autoencoder.save(autoencoder_model_path)
encoder.save(encoder_model_path)
print(f"Autoencoder saved at: {autoencoder_model_path}")
print(f"Encoder saved at: {encoder_model_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 793 images belonging to 2 classes.
Epoch 1/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 20s 39ms/step - loss: 0.0174
Epoch 2/15
  5/200 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - loss: 0.0083

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 0.0057
Epoch 3/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - loss: 0.0045
Epoch 4/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 0.0045
Epoch 5/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - loss: 0.0046
Epoch 6/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - loss: 0.0040
Epoch 7/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 0.0037
Epoch 8/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - loss: 0.0037
Epoch 9/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - loss: 0.0039
Epoch 10/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - loss: 0.0033
Epoch 11/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 0.0036
Epoch 12/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 0.0032
Epoch 13/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - loss: 0.0033
Epoch 14/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 0.0033
Epoch 15/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 0.0030
Autoencoder saved at: /content/drive/MyDrive/skin_can

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define paths
combined_folder = '/content/drive/MyDrive/skin_cancer_images/combined'
encoder_model_path = '/content/drive/MyDrive/skin_cancer/improved_encoder.keras'
output_csv_path = '/content/drive/MyDrive/skin_cancer/improved_autoencoder_features_with_labels.csv'

# Load the trained encoder model
encoder = load_model(encoder_model_path)

# Create ImageDataGenerator for loading images (without augmentation)
datagen = ImageDataGenerator(rescale=1./255)
data_generator = datagen.flow_from_directory(
    combined_folder,
    target_size=(160, 160),  # Ensure this matches the training image size
    batch_size=1,  # Process one image at a time to avoid memory issues
    class_mode=None,  # No labels needed as we extract features
    shuffle=False  # Maintain order for correct label mapping
)

# Extract labels (folder names represent class labels)
class_indices = data_generator.class_indices
label_map = {v: k for k, v in class_indices.items()}  # Convert index to class name

# Extract features
features_list = []
labels_list = []

for i in range(len(data_generator)):
    image = data_generator[i]  # Get image
    feature_vector = encoder.predict(image, verbose=0).flatten()  # Extract features
    label_index = data_generator.classes[i]  # Get corresponding label index
    label_name = label_map[label_index]  # Convert index to class name

    features_list.append(feature_vector)
    labels_list.append(label_name)

# Convert to Pandas DataFrame
feature_columns = [f'feature_{i}' for i in range(len(features_list[0]))]
df = pd.DataFrame(features_list, columns=feature_columns)
df['label'] = labels_list  # Add label column

# Save to CSV
df.to_csv(output_csv_path, index=False)
print(f"Features saved successfully at: {output_csv_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 793 images belonging to 2 classes.
Features saved successfully at: /content/drive/MyDrive/skin_cancer/improved_autoencoder_features_with_labels.csv


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Path to extracted features CSV
csv_path = '/content/drive/MyDrive/skin_cancer/l_improved_autoencoder_features_with_labels.csv'

# Load features and labels
df = pd.read_csv(csv_path)
X = df.drop(columns=['label']).values  # Features
y = df['label'].values  # Labels

# Split dataset into train and test (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Standardize the features for better performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize classifiers
svm_model = SVC(kernel='rbf', C=1.0, random_state=42)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
nn_model = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=500, random_state=42)

# Train models
svm_model.fit(X_train, y_train)
rf_model.fit(X_train, y_train)
nn_model.fit(X_train, y_train)

# Make predictions
y_pred_svm = svm_model.predict(X_test)
y_pred_rf = rf_model.predict(X_test)
y_pred_nn = nn_model.predict(X_test)

# Calculate accuracy
accuracy_svm = accuracy_score(y_test, y_pred_svm)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
accuracy_nn = accuracy_score(y_test, y_pred_nn)

# Print results
print(f"SVM Accuracy: {accuracy_svm:.4f}")
print(f"Random Forest Accuracy: {accuracy_rf:.4f}")
print(f"Neural Network Accuracy: {accuracy_nn:.4f}")


SVM Accuracy: 0.7421
Random Forest Accuracy: 0.7925
Neural Network Accuracy: 0.7799


In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define paths
combined_folder = '/content/drive/MyDrive/skin_cancer_images/combined'
encoder_model_path = '/content/drive/MyDrive/skin_cancer/improved_encoder.keras'
output_csv_path = '/content/drive/MyDrive/skin_cancer/l_improved_autoencoder_features_with_labels.csv'

# Load the trained encoder model
encoder = load_model(encoder_model_path)

# Identify the correct bottleneck layer
bottleneck_layer_name = 'encoder_output'  # Change this if needed
bottleneck_model = Model(inputs=encoder.input, outputs=encoder.get_layer(bottleneck_layer_name).output)

# Create ImageDataGenerator for loading images (without augmentation)
datagen = ImageDataGenerator(rescale=1./255)
data_generator = datagen.flow_from_directory(
    combined_folder,
    target_size=(160, 160),  # Ensure this matches the training image size
    batch_size=1,  # Process one image at a time to avoid memory issues
    class_mode=None,  # No labels needed as we extract features
    shuffle=False  # Maintain order for correct label mapping
)

# Extract labels (folder names represent class labels)
class_indices = data_generator.class_indices
label_map = {v: k for k, v in class_indices.items()}  # Convert index to class name

# Extract features
features_list = []
labels_list = []

for i in range(len(data_generator)):
    image = data_generator[i]  # Get image
    feature_vector = bottleneck_model.predict(image, verbose=0).flatten()  # Extract bottleneck features
    label_index = data_generator.classes[i]  # Get corresponding label index
    label_name = label_map[label_index]  # Convert index to class name

    features_list.append(feature_vector)
    labels_list.append(label_name)

# Convert to Pandas DataFrame
feature_columns = [f'feature_{i}' for i in range(len(features_list[0]))]
df = pd.DataFrame(features_list, columns=feature_columns)
df['label'] = labels_list  # Add label column

# Save to CSV
df.to_csv(output_csv_path, index=False)
print(f"Latent features saved successfully at: {output_csv_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 793 images belonging to 2 classes.
Latent features saved successfully at: /content/drive/MyDrive/skin_cancer/l_improved_autoencoder_features_with_labels.csv


### Concatenate resnet and autoencoder

### no pca

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load extracted features from CSVs
autoencoder_features = pd.read_csv('/content/drive/MyDrive/skin_cancer/improved_autoencoder_features_with_labels.csv')
resnet_features = pd.read_csv('/content/drive/MyDrive/skin_cancer/resnet50_conv5_block2_out_features_with_labels.csv')

# Convert autoencoder labels to match ResNet format (1.0 for benign, 0.0 for malignant)
label_mapping = {'benign': 1.0, 'malignant': 0.0}
autoencoder_features['label'] = autoencoder_features['label'].map(label_mapping)

# Ensure labels are consistent across both datasets
y = autoencoder_features['label'].values  # Both feature sets now have 1.0 for benign, 0.0 for malignant

# Drop label columns to extract feature matrices
X_auto = autoencoder_features.drop(columns=['label']).values
X_resnet = resnet_features.drop(columns=['label']).values

# Normalize features
scaler = StandardScaler()
X_auto_scaled = scaler.fit_transform(X_auto)
X_resnet_scaled = scaler.fit_transform(X_resnet)

# Concatenate the two feature sets
X_combined = np.concatenate((X_auto_scaled, X_resnet_scaled), axis=1)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Train classifiers
svm = SVC(kernel='rbf', C=1.0)
rf = RandomForestClassifier(n_estimators=100)
nn = MLPClassifier(hidden_layer_sizes=(128, 64), activation='relu', max_iter=1000)

# Train models
svm.fit(X_train, y_train)
rf.fit(X_train, y_train)
nn.fit(X_train, y_train)

# Evaluate models
y_pred_svm = svm.predict(X_test)
y_pred_rf = rf.predict(X_test)
y_pred_nn = nn.predict(X_test)

print("SVM Accuracy:", accuracy_score(y_test, y_pred_svm))
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Neural Network Accuracy:", accuracy_score(y_test, y_pred_nn))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### PCA

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load extracted features from CSVs
autoencoder_features = pd.read_csv('/content/drive/MyDrive/skin_cancer/improved_autoencoder_features_with_labels.csv')
resnet_features = pd.read_csv('/content/drive/MyDrive/skin_cancer/resnet50_conv5_block2_out_features_with_labels.csv')

# Convert autoencoder labels to match ResNet format (1.0 for benign, 0.0 for malignant)
label_mapping = {'benign': 1.0, 'malignant': 0.0}
autoencoder_features['label'] = autoencoder_features['label'].map(label_mapping)

# Ensure labels are consistent across both datasets
y = autoencoder_features['label'].values  # Now both feature sets have 1.0 for benign, 0.0 for malignant

# Drop label columns to extract feature matrices
X_auto = autoencoder_features.drop(columns=['label']).values
X_resnet = resnet_features.drop(columns=['label']).values

# Normalize features
scaler = StandardScaler()
X_auto_scaled = scaler.fit_transform(X_auto)
X_resnet_scaled = scaler.fit_transform(X_resnet)

# Apply PCA to reduce feature dimensions
pca_auto = PCA(n_components=100)  # Adjust based on variance explained
pca_resnet = PCA(n_components=100)

X_auto_pca = pca_auto.fit_transform(X_auto_scaled)
X_resnet_pca = pca_resnet.fit_transform(X_resnet_scaled)

# Concatenate the reduced feature sets
X_combined = np.concatenate((X_auto_pca, X_resnet_pca), axis=1)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Train classifiers
svm = SVC(kernel='rbf', C=1.0)
rf = RandomForestClassifier(n_estimators=100)
nn = MLPClassifier(hidden_layer_sizes=(128, 64), activation='relu', max_iter=1000)

# Train models
svm.fit(X_train, y_train)
rf.fit(X_train, y_train)
nn.fit(X_train, y_train)

# Evaluate models
y_pred_svm = svm.predict(X_test)
y_pred_rf = rf.predict(X_test)
y_pred_nn = nn.predict(X_test)

print("SVM Accuracy:", accuracy_score(y_test, y_pred_svm))
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Neural Network Accuracy:", accuracy_score(y_test, y_pred_nn))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
SVM Accuracy: 0.7735849056603774
Random Forest Accuracy: 0.7735849056603774
Neural Network Accuracy: 0.7610062893081762


### LDA

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load extracted features from CSVs
autoencoder_features = pd.read_csv('/content/drive/MyDrive/skin_cancer/improved_autoencoder_features_with_labels.csv')
resnet_features = pd.read_csv('/content/drive/MyDrive/skin_cancer/resnet50_conv5_block2_out_features_with_labels.csv')

# Convert autoencoder labels to match ResNet format (1.0 for benign, 0.0 for malignant)
label_mapping = {'benign': 1.0, 'malignant': 0.0}
autoencoder_features['label'] = autoencoder_features['label'].map(label_mapping)

# Ensure labels are consistent across both datasets
y = autoencoder_features['label'].values  # Now both feature sets have 1.0 for benign, 0.0 for malignant

# Drop label columns to extract feature matrices
X_auto = autoencoder_features.drop(columns=['label']).values
X_resnet = resnet_features.drop(columns=['label']).values

# Normalize features
scaler = StandardScaler()
X_auto_scaled = scaler.fit_transform(X_auto)
X_resnet_scaled = scaler.fit_transform(X_resnet)

# Concatenate both feature sets
X_combined = np.concatenate((X_auto_scaled, X_resnet_scaled), axis=1)

# Apply LDA to reduce dimensions (LDA components <= number of classes - 1, so here 1 component)
lda = LDA(n_components=1)
X_lda = lda.fit_transform(X_combined, y)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_lda, y, test_size=0.2, random_state=42)

# Train classifiers
svm = SVC(kernel='rbf', C=1.0)
rf = RandomForestClassifier(n_estimators=100)
nn = MLPClassifier(hidden_layer_sizes=(128, 64), activation='relu', max_iter=1000)

# Train models
svm.fit(X_train, y_train)
rf.fit(X_train, y_train)
nn.fit(X_train, y_train)

# Evaluate models
y_pred_svm = svm.predict(X_test)
y_pred_rf = rf.predict(X_test)
y_pred_nn = nn.predict(X_test)

print("SVM Accuracy:", accuracy_score(y_test, y_pred_svm))
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Neural Network Accuracy:", accuracy_score(y_test, y_pred_nn))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### umap

In [ ]:
import numpy as np
import pandas as pd
import torch  # GPU Acceleration
import gc  # Garbage Collection to Free RAM
from cuml import UMAP  # GPU-accelerated UMAP
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from google.colab import drive

# 🚀 Check GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# 🔗 Mount Google Drive
drive.mount('/content/drive')

# 📂 Load extracted features
autoencoder_features = pd.read_csv('/content/drive/MyDrive/skin_cancer/improved_autoencoder_features_with_labels.csv')
resnet_features = pd.read_csv('/content/drive/MyDrive/skin_cancer/resnet50_conv5_block2_out_features_with_labels.csv')

# 🔄 Convert labels
label_mapping = {'benign': 1.0, 'malignant': 0.0}
autoencoder_features['label'] = autoencoder_features['label'].map(label_mapping)
y = autoencoder_features['label'].values  # Extract labels

# 📊 Extract feature matrices
X_auto = autoencoder_features.drop(columns=['label']).values
X_resnet = resnet_features.drop(columns=['label']).values

# 📏 Normalize features
scaler = StandardScaler()
X_auto_scaled = scaler.fit_transform(X_auto)
X_resnet_scaled = scaler.fit_transform(X_resnet)

# 🔗 Combine features
X_combined = np.concatenate((X_auto_scaled, X_resnet_scaled), axis=1)

# 🧠 **Apply GPU-accelerated UMAP for Feature Reduction**
umap = UMAP(n_components=50, random_state=42)  # Reduce to 50 features
X_umap = umap.fit_transform(X_combined)

# 🚮 Free RAM
del X_auto, X_resnet, X_auto_scaled, X_resnet_scaled, X_combined
gc.collect()

# 🛑 **Fixing Data Leakage**
X_train, X_test, y_train, y_test = train_test_split(
    X_umap, y, test_size=0.2, random_state=42, shuffle=True, stratify=y
)

# 🚨 Data Check
print(f"Train Shape: {X_train.shape}, Test Shape: {X_test.shape}")

# 🔥 Train Classifiers
svm = SVC(kernel='rbf', C=1.0)
rf = RandomForestClassifier(n_estimators=100)

svm.fit(X_train, y_train)
rf.fit(X_train, y_train)

# 🎯 Predictions
y_pred_svm = svm.predict(X_test)
y_pred_rf = rf.predict(X_test)

# 📊 Accuracy Results
print("SVM Accuracy:", accuracy_score(y_test, y_pred_svm))
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))

# 🧠 **GPU-based Neural Network using PyTorch**
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)

# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# 🔥 Define Neural Network (GPU Accelerated)
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(50, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

# 🚀 Move Model to GPU
model = SimpleNN().to(device)

# 🎯 Loss & Optimizer
criterion = nn.BCELoss()  # Binary Classification Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 🏋️ Train Model
epochs = 10
for epoch in range(epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch).squeeze()
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

# 📊 Evaluate Model
model.eval()
y_pred_nn = model(X_test_tensor).cpu().detach().numpy().round()
print("Neural Network Accuracy:", accuracy_score(y_test, y_pred_nn))


Using device: cuda
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Train Shape: (634, 50), Test Shape: (159, 50)
SVM Accuracy: 0.6981132075471698
Random Forest Accuracy: 0.7421383647798742
Epoch 1/10, Loss: 0.7019618153572083
Epoch 2/10, Loss: 0.6786752939224243
Epoch 3/10, Loss: 0.7452600598335266
Epoch 4/10, Loss: 0.6410877108573914
Epoch 5/10, Loss: 0.6243048906326294
Epoch 6/10, Loss: 0.665518581867218
Epoch 7/10, Loss: 0.4798668622970581
Epoch 8/10, Loss: 0.5306162238121033
Epoch 9/10, Loss: 0.5745664238929749
Epoch 10/10, Loss: 0.4457313120365143
Neural Network Accuracy: 0.7232704402515723


### XGB

In [ ]:
import numpy as np
import pandas as pd
import torch  # For GPU checks
import gc  # Garbage Collection to Free RAM
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier  # GPU-accelerated XGBoost
from google.colab import drive

# 🚀 Check GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# 🔗 Mount Google Drive
drive.mount('/content/drive')

# 📂 Load extracted features
autoencoder_features = pd.read_csv('/content/drive/MyDrive/skin_cancer/improved_autoencoder_features_with_labels.csv')
resnet_features = pd.read_csv('/content/drive/MyDrive/skin_cancer/resnet50_conv5_block2_out_features_with_labels.csv')

# 🔄 Convert labels
label_mapping = {'benign': 1.0, 'malignant': 0.0}
autoencoder_features['label'] = autoencoder_features['label'].map(label_mapping)
y = autoencoder_features['label'].values  # Extract labels

# 📊 Extract feature matrices
X_auto = autoencoder_features.drop(columns=['label']).values
X_resnet = resnet_features.drop(columns=['label']).values

# 📏 Normalize features
scaler = StandardScaler()
X_auto_scaled = scaler.fit_transform(X_auto)
X_resnet_scaled = scaler.fit_transform(X_resnet)

# 🔗 Combine features
X_combined = np.concatenate((X_auto_scaled, X_resnet_scaled), axis=1)

# 🏆 **Feature Selection Using Mutual Information**
num_features = 100  # Keep only the top 100 most important features
mi_scores = mutual_info_classif(X_combined, y)
top_features_idx = np.argsort(mi_scores)[-num_features:]  # Select top N features
X_selected = X_combined[:, top_features_idx]

# 🚮 Free RAM
del X_auto, X_resnet, X_auto_scaled, X_resnet_scaled, X_combined
gc.collect()

# 🛑 **Fixing Data Leakage**
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.2, random_state=42, shuffle=True, stratify=y
)

# 🚨 Data Check
print(f"Train Shape: {X_train.shape}, Test Shape: {X_test.shape}")

# 🔥 **Train GPU-Accelerated XGBoost Classifier**
xgb = XGBClassifier(
    tree_method='gpu_hist',  # Enable GPU acceleration
    predictor='gpu_predictor',
    n_estimators=200,
    learning_rate=0.1,
    max_depth=5
)

xgb.fit(X_train, y_train)

# 🎯 Predictions
y_pred_xgb = xgb.predict(X_test)

# 📊 Accuracy Results
print("XGBoost Accuracy:", accuracy_score(y_test, y_pred_xgb))


Using device: cuda
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Check if X_train and X_test have overlapping values
print("Overlap in train & test:", np.intersect1d(X_train.cpu().numpy(), X_test.cpu().numpy()).size > 0)


Overlap in train & test: True


In [ ]:
import pandas as pd

# Load the feature files
autoencoder_features = pd.read_csv('/content/drive/MyDrive/skin_cancer/improved_autoencoder_features_with_labels.csv')
resnet_features = pd.read_csv('/content/drive/MyDrive/skin_cancer/resnet50_conv5_block2_out_features_with_labels.csv')

# Print first few rows of the labels column
print("Autoencoder Labels:")
print(autoencoder_features['label'].head())  # Assuming 'label' is the column name

print("\nResNet50 Labels:")
print(resnet_features['label'].head())  # Assuming 'label' is the column name


Autoencoder Labels:
0    benign
1    benign
2    benign
3    benign
4    benign
Name: label, dtype: object

ResNet50 Labels:
0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
Name: label, dtype: float64


### Improve 2.0

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.decomposition import PCA
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Paths
encoder_model_path = '/content/drive/MyDrive/skin_cancer/improved_encoder.keras'
combined_folder = '/content/drive/MyDrive/skin_cancer_images/combined'
output_csv_path = '/content/drive/MyDrive/skin_cancer/advanced_features.csv'

# Load the trained encoder model
encoder = load_model(encoder_model_path)

# Extract the last layer (bottleneck)
bottleneck_layer = encoder.get_layer("encoder_output").output
feature_extractor = Model(inputs=encoder.input, outputs=bottleneck_layer)

# Image Generator
datagen = ImageDataGenerator(rescale=1./255)
data_generator = datagen.flow_from_directory(
    combined_folder, target_size=(160, 160), batch_size=1, class_mode=None, shuffle=False
)

# Map class indices to labels
label_map = {v: k for k, v in data_generator.class_indices.items()}

# Extract features
features_list = []
labels_list = []

for i in range(len(data_generator)):
    image = data_generator[i]
    feature_vector = feature_extractor.predict(image, verbose=0).flatten()  # Extract bottleneck features

    label_name = label_map[data_generator.classes[i]]

    features_list.append(feature_vector)
    labels_list.append(label_name)

# Convert to DataFrame
feature_columns = [f'feature_{i}' for i in range(len(features_list[0]))]
df = pd.DataFrame(features_list, columns=feature_columns)
df['label'] = labels_list

# Apply PCA to reduce dimensionality
pca = PCA(n_components=100)  # Reduce to 100 principal features
pca_features = pca.fit_transform(df.iloc[:, :-1])
df_pca = pd.DataFrame(pca_features, columns=[f'pca_{i}' for i in range(100)])
df_pca['label'] = labels_list

# Save to CSV
df_pca.to_csv(output_csv_path, index=False)
print(f"Optimized features saved at: {output_csv_path}")


Mounted at /content/drive
Found 793 images belonging to 2 classes.
Optimized features saved at: /content/drive/MyDrive/skin_cancer/advanced_features.csv


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score

# Load the extracted features CSV
csv_path = '/content/drive/MyDrive/skin_cancer/advanced_features.csv'
df = pd.read_csv(csv_path)

# Separate features and labels
X = df.iloc[:, :-1].values  # All feature columns
y = df['label'].values  # Labels

# Encode labels (convert categorical to numeric)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split dataset into training & testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# --- Train SVM Classifier ---
svm_model = SVC(kernel='rbf', C=10, gamma='scale')  # Tuned hyperparameters
svm_model.fit(X_train, y_train)
svm_preds = svm_model.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_preds)
print(f"SVM Accuracy: {svm_accuracy:.4f}")

# --- Train Random Forest Classifier ---
rf_model = RandomForestClassifier(n_estimators=200, max_depth=20, random_state=42)
rf_model.fit(X_train, y_train)
rf_preds = rf_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_preds)
print(f"Random Forest Accuracy: {rf_accuracy:.4f}")

# --- Train Neural Network (MLP) ---
nn_model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(len(np.unique(y_encoded)), activation='softmax')  # Output layer
])

nn_model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
nn_model.fit(X_train, y_train, epochs=30, batch_size=16, verbose=1, validation_data=(X_test, y_test))

nn_loss, nn_accuracy = nn_model.evaluate(X_test, y_test, verbose=0)
print(f"Neural Network Accuracy: {nn_accuracy:.4f}")


SVM Accuracy: 0.7296
Random Forest Accuracy: 0.7484


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 65ms/step - accuracy: 0.5456 - loss: 0.7852 - val_accuracy: 0.6164 - val_loss: 0.6479
Epoch 2/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6464 - loss: 0.6197 - val_accuracy: 0.6918 - val_loss: 0.6219
Epoch 3/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7474 - loss: 0.5395 - val_accuracy: 0.7170 - val_loss: 0.5759
Epoch 4/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7832 - loss: 0.4582 - val_accuracy: 0.7358 - val_loss: 0.5880
Epoch 5/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8528 - loss: 0.3833 - val_accuracy: 0.7233 - val_loss: 0.5816
Epoch 6/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8635 - loss: 0.3447 - val_accuracy: 0.7296 - val_loss: 0.6077
Epoch 7/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8819 - loss: 0.3103 - val_accuracy: 0.7296 - val_loss: 0.6231
Epoch 8/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9102 - loss: 0.2697 - val_accuracy: 0.7421 - val_loss

### Improve 3.0

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Conv2D, MaxPooling2D, UpSampling2D,
                                     BatchNormalization, Dropout, GaussianNoise, Flatten, Dense, Reshape)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive

# 🚀 Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# 🗂 Define dataset paths
combined_folder = '/content/drive/MyDrive/skin_cancer_images/combined'
autoencoder_model_path = '/content/drive/MyDrive/skin_cancer/improved2_autoencoder.keras'
encoder_model_path = '/content/drive/MyDrive/skin_cancer/improved2_encoder.keras'

# 🚀 Clear GPU memory before training
tf.keras.backend.clear_session()

# 🎯 Define input image shape
input_shape = (160, 160, 3)

# 🎭 Encoder
input_img = Input(shape=input_shape)
x = GaussianNoise(0.1)(input_img)  # Adding noise for denoising AE
x = Conv2D(32, (3,3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2), padding='same')(x)  # 160 → 80

x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2), padding='same')(x)  # 80 → 40

x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2), padding='same')(x)  # 40 → 20

x = Conv2D(256, (3,3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2), padding='same')(x)  # 20 → 10

x = Flatten()(x)
bottleneck = Dense(512, activation='relu', name="encoder_output")(x)  # Only 512 neurons here

# 🎭 Full Decoder (Fixed)
x = Dense(10 * 10 * 256, activation='relu')(bottleneck)  # Expanding back
x = Reshape((10, 10, 256))(x)

x = Conv2D(256, (3,3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = UpSampling2D((2,2))(x)  # 10 → 20

x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = UpSampling2D((2,2))(x)  # 20 → 40

x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = UpSampling2D((2,2))(x)  # 40 → 80

x = Conv2D(32, (3,3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = UpSampling2D((2,2))(x)  # 80 → 160

output_img = Conv2D(3, (3,3), activation='sigmoid', padding='same')(x)  # Final Output

# ✅ Build Autoencoder Model
autoencoder = Model(input_img, output_img)
autoencoder.compile(optimizer='adam', loss='mse')

# ✅ Extract Encoder Model
encoder = Model(input_img, bottleneck)

# 🔄 Create ImageDataGenerator for loading images
datagen = ImageDataGenerator(rescale=1./255)
data_generator = datagen.flow_from_directory(
    combined_folder,
    target_size=(160, 160),
    batch_size=8,
    class_mode='input',
    shuffle=True
)

autoencoder.fit(
    data_generator,
    epochs=20,
    steps_per_epoch=len(data_generator) // 8
)

# 💾 Save Models
autoencoder.save(autoencoder_model_path)
encoder.save(encoder_model_path)
print(f"Autoencoder saved at: {autoencoder_model_path}")
print(f"Encoder saved at: {encoder_model_path}")


Mounted at /content/drive
Found 793 images belonging to 2 classes.
Epoch 1/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 20s 40ms/step - loss: 0.0561
Epoch 2/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.0220
Epoch 3/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 8s 709ms/step - loss: 0.0168
Epoch 4/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.0173
Epoch 5/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.0143
Epoch 6/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.0144
Epoch 7/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.0128
Epoch 8/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.0098
Epoch 9/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0075
Epoch 10/20
 2/12 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.0133

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.0127
Epoch 11/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.0109
Epoch 12/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.0083
Epoch 13/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.0095
Epoch 14/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.0100
Epoch 15/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.0090
Epoch 16/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.0069
Epoch 17/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0105
Epoch 18/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0073
Epoch 19/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 0.0100
Epoch 20/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0073
Autoencoder saved at: /content/drive/MyDrive/skin_cancer/improved2_autoencoder.keras
Encoder saved at: /content/drive/MyDrive/skin_cancer/improved2_encoder.keras


In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths
encoder_model_path = '/content/drive/MyDrive/skin_cancer/improved2_encoder.keras'
combined_folder = '/content/drive/MyDrive/skin_cancer_images/combined'
csv_output_path = '/content/drive/MyDrive/skin_cancer/improved2_extracted_features.csv'

# Load trained encoder
encoder = tf.keras.models.load_model(encoder_model_path)

# Create ImageDataGenerator for loading images
datagen = ImageDataGenerator(rescale=1./255)
data_generator = datagen.flow_from_directory(
    combined_folder,
    target_size=(160, 160),
    batch_size=1,  # Process one image at a time
    class_mode='binary',  # Ensure labels are available
    shuffle=False
)

# Extract labels correctly
labels = data_generator.classes  # Get numeric labels
class_indices = data_generator.class_indices  # Mapping class names to indices
index_to_class = {v: k for k, v in class_indices.items()}  # Reverse mapping
labels = [index_to_class[label] for label in labels]  # Convert to class names

# Extract features
features = encoder.predict(data_generator)

# Convert features to DataFrame
df_features = pd.DataFrame(features)
df_features['label'] = labels  # Add labels column

# Save to CSV
df_features.to_csv(csv_output_path, index=False)
print(f"Features saved at: {csv_output_path}")


Found 793 images belonging to 2 classes.
793/793 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step
Features saved at: /content/drive/MyDrive/skin_cancer/improved2_extracted_features.csv


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load extracted features
csv_path = '/content/drive/MyDrive/skin_cancer/improved2_extracted_features.csv'
df = pd.read_csv(csv_path)

# Separate features and labels
X = df.drop(columns=['label']).values  # Features
y = df['label'].values  # Labels

# Encode labels (convert class names to numbers)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)  # Converts 'Benign' -> 0, 'Malignant' -> 1

# Split dataset into training and testing (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train & Evaluate SVM
svm_model = SVC(kernel='rbf', C=1.0)
svm_model.fit(X_train, y_train)
svm_preds = svm_model.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_preds)
print(f"🔹 SVM Accuracy: {svm_accuracy:.4f}")

# Train & Evaluate Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_preds = rf_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_preds)
print(f"🔹 Random Forest Accuracy: {rf_accuracy:.4f}")

# Train Neural Network (Simple Feedforward)
nn_model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification
])

nn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the neural network
nn_model.fit(X_train, y_train, epochs=20, batch_size=8, verbose=1, validation_data=(X_test, y_test))

# Evaluate Neural Network
_, nn_accuracy = nn_model.evaluate(X_test, y_test)
print(f"🔹 Neural Network Accuracy: {nn_accuracy:.4f}")


🔹 SVM Accuracy: 0.5912
🔹 Random Forest Accuracy: 0.6792
Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


80/80 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.5311 - loss: 6.8927 - val_accuracy: 0.5660 - val_loss: 1.3649
Epoch 2/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5775 - loss: 2.9498 - val_accuracy: 0.5849 - val_loss: 1.1321
Epoch 3/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5766 - loss: 0.9359 - val_accuracy: 0.5157 - val_loss: 1.2940
Epoch 4/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6368 - loss: 0.9896 - val_accuracy: 0.5975 - val_loss: 0.7004
Epoch 5/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6224 - loss: 1.0504 - val_accuracy: 0.5786 - val_loss: 1.0449
Epoch 6/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6416 - loss: 0.7604 - val_accuracy: 0.6289 - val_loss: 0.6915
Epoch 7/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6487 - loss: 0.6631 - val_accuracy: 0.6289 - val_loss: 0.6835
Epoch 8/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6116 - loss: 0.6638 - val_accuracy: 0.6038 - val_loss: 1.0796
Ep

### IMPROVE 4.0

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Conv2D, MaxPooling2D, UpSampling2D,
                                     BatchNormalization, Dropout, GaussianNoise, Flatten, Dense)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define paths
combined_folder = '/content/drive/MyDrive/skin_cancer_images/combined'
autoencoder_model_path = '/content/drive/MyDrive/skin_cancer/improved3_autoencoder.keras'
encoder_model_path = '/content/drive/MyDrive/skin_cancer/improved3_encoder.keras'

# Clear GPU memory before training
tf.keras.backend.clear_session()

# Autoencoder Architecture
input_shape = (160, 160, 3)

# Encoder
input_img = Input(shape=input_shape)
x = GaussianNoise(0.1)(input_img)  # Adding noise for denoising AE
x = Conv2D(32, (3,3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2), padding='same')(x)

x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2), padding='same')(x)

x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2), padding='same')(x)

x = Conv2D(256, (3,3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2), padding='same')(x)

x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)  # Dropout for regularization
bottleneck = Dense(256, activation='relu', name="encoder_output")(x)

# Decoder
x = Dense(512, activation='relu')(bottleneck)
x = Dropout(0.3)(x)
x = Dense(160*160*3, activation='sigmoid')(x)
x = tf.keras.layers.Reshape((160, 160, 3))(x)

# Build Autoencoder Model
autoencoder = Model(input_img, x)
autoencoder.compile(optimizer='adam', loss='mse')

# Extract Encoder Model
encoder = Model(input_img, bottleneck)

# Create ImageDataGenerator for loading images
datagen = ImageDataGenerator(rescale=1./255)
data_generator = datagen.flow_from_directory(
    combined_folder,
    target_size=(160, 160),
    batch_size=8,
    class_mode='input',
    shuffle=True
)

# Train Autoencoder
autoencoder.fit(
    data_generator,
    epochs=20,  # Slightly increased for better feature learning
    steps_per_epoch=200
)

# Save Models
autoencoder.save(autoencoder_model_path)
encoder.save(encoder_model_path)
print(f"Autoencoder saved at: {autoencoder_model_path}")
print(f"Encoder saved at: {encoder_model_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 793 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 19s 41ms/step - loss: 0.0151
Epoch 2/20


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


200/200 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - loss: 0.0099
Epoch 3/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - loss: 0.0094
Epoch 4/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 0.0093
Epoch 5/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - loss: 0.0099
Epoch 6/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - loss: 0.0097
Epoch 7/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - loss: 0.0078
Epoch 8/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 0.0078
Epoch 9/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - loss: 0.0073
Epoch 10/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - loss: 0.0081
Epoch 11/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 0.0078
Epoch 12/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 0.0082
Epoch 13/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - loss: 0.0073
Epoch 14/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - loss: 0.0075
Epoch 15/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 0.0067
Epoch 16/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths
encoder_model_path = '/content/drive/MyDrive/skin_cancer/improved3_encoder.keras'
combined_folder = '/content/drive/MyDrive/skin_cancer_images/combined'
csv_output_path = '/content/drive/MyDrive/skin_cancer/improved3_extracted_features.csv'

# Load trained encoder
encoder = tf.keras.models.load_model(encoder_model_path)

# Create ImageDataGenerator for loading images
datagen = ImageDataGenerator(rescale=1./255)
data_generator = datagen.flow_from_directory(
    combined_folder,
    target_size=(160, 160),
    batch_size=1,  # Process one image at a time
    class_mode='binary',  # Ensure labels are available
    shuffle=False
)

# Extract labels correctly
labels = data_generator.classes  # Get numeric labels
class_indices = data_generator.class_indices  # Mapping class names to indices
index_to_class = {v: k for k, v in class_indices.items()}  # Reverse mapping
labels = [index_to_class[label] for label in labels]  # Convert to class names

# Extract features
features = encoder.predict(data_generator)

# Convert features to DataFrame
df_features = pd.DataFrame(features)
df_features['label'] = labels  # Add labels column

# Save to CSV
df_features.to_csv(csv_output_path, index=False)
print(f"Features saved at: {csv_output_path}")


Found 793 images belonging to 2 classes.
793/793 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step
Features saved at: /content/drive/MyDrive/skin_cancer/improved3_extracted_features.csv


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load extracted features
csv_path = '/content/drive/MyDrive/skin_cancer/improved3_extracted_features.csv'
df = pd.read_csv(csv_path)

# Separate features and labels
X = df.drop(columns=['label']).values  # Features
y = df['label'].values  # Labels

# Encode labels (convert class names to numbers)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)  # Converts 'Benign' -> 0, 'Malignant' -> 1

# Split dataset into training and testing (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train & Evaluate SVM
svm_model = SVC(kernel='rbf', C=1.0)
svm_model.fit(X_train, y_train)
svm_preds = svm_model.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_preds)
print(f"🔹 SVM Accuracy: {svm_accuracy:.4f}")

# Train & Evaluate Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_preds = rf_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_preds)
print(f"🔹 Random Forest Accuracy: {rf_accuracy:.4f}")

# Train Neural Network (Simple Feedforward)
nn_model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification
])

nn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the neural network
nn_model.fit(X_train, y_train, epochs=20, batch_size=8, verbose=1, validation_data=(X_test, y_test))

# Evaluate Neural Network
_, nn_accuracy = nn_model.evaluate(X_test, y_test)
print(f"🔹 Neural Network Accuracy: {nn_accuracy:.4f}")


🔹 SVM Accuracy: 0.5472
🔹 Random Forest Accuracy: 0.5031
Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


80/80 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.5453 - loss: 0.8748 - val_accuracy: 0.6164 - val_loss: 0.7945
Epoch 2/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5157 - loss: 0.7780 - val_accuracy: 0.5849 - val_loss: 0.8902
Epoch 3/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5836 - loss: 0.7886 - val_accuracy: 0.6038 - val_loss: 0.6711
Epoch 4/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6255 - loss: 0.6773 - val_accuracy: 0.5723 - val_loss: 0.7055
Epoch 5/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6629 - loss: 0.6135 - val_accuracy: 0.5094 - val_loss: 0.7075
Epoch 6/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6072 - loss: 0.6616 - val_accuracy: 0.5912 - val_loss: 0.6817
Epoch 7/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6358 - loss: 0.6189 - val_accuracy: 0.5220 - val_loss: 0.7128
Epoch 8/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5994 - loss: 0.6229 - val_accuracy: 0.5660 - val_loss: 0.7500
Ep

### DEEP ENCODER

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Add
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive

# 🚀 Mount Google Drive
drive.mount('/content/drive')

# 📂 Define paths
combined_folder = '/content/drive/MyDrive/skin_cancer_images/combined'
autoencoder_model_path = '/content/drive/MyDrive/skin_cancer/deep_autoencoder.keras'
encoder_model_path = '/content/drive/MyDrive/skin_cancer/deep_encoder.keras'

# 🧹 Clear GPU memory
tf.keras.backend.clear_session()

# 📌 Autoencoder Architecture (Improved with Residual Connections)
input_shape = (160, 160, 3)  # Keeping image size 160x160

# 🔹 Encoder
input_img = Input(shape=input_shape)
x = Conv2D(32, (3,3), activation='relu', padding='same')(input_img)
x = BatchNormalization()(x)
x = Conv2D(32, (3,3), activation='relu', padding='same')(x)  # Extra Conv Layer
x = MaxPooling2D((2,2), padding='same')(x)

x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(64, (3,3), activation='relu', padding='same')(x)  # Extra Conv Layer
x = MaxPooling2D((2,2), padding='same')(x)

x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
residual = x  # Store for residual connection
x = Conv2D(128, (3,3), activation='relu', padding='same')(x)  # Extra Conv Layer
x = Add()([x, residual])  # 🔄 Residual Connection
x = MaxPooling2D((2,2), padding='same')(x)

bottleneck = Conv2D(256, (3,3), activation='relu', padding='same', name="encoder_output")(x)

# 🔹 Decoder
x = Conv2D(128, (3,3), activation='relu', padding='same')(bottleneck)
x = BatchNormalization()(x)
x = UpSampling2D((2,2))(x)

x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = UpSampling2D((2,2))(x)

x = Conv2D(32, (3,3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = UpSampling2D((2,2))(x)

output_img = Conv2D(3, (3,3), activation='sigmoid', padding='same')(x)

# 🔹 Build Autoencoder Model
autoencoder = Model(input_img, output_img)
autoencoder.compile(optimizer='adam', loss='mse')

# 🔹 Extract Encoder Model
encoder = Model(input_img, bottleneck)

# 🔄 Create ImageDataGenerator for loading images
datagen = ImageDataGenerator(rescale=1./255)
data_generator = datagen.flow_from_directory(
    combined_folder,
    target_size=(160, 160),
    batch_size=16,  # 🚀 Increased Batch Size
    class_mode='input',
    shuffle=True
)

# 🔥 Train Autoencoder
autoencoder.fit(
    data_generator,
    epochs=20,  # 🚀 Increased epochs
    steps_per_epoch=250  # 🚀 More steps per epoch
)

# 💾 Save Models
autoencoder.save(autoencoder_model_path)
encoder.save(encoder_model_path)
print(f"✅ Autoencoder saved at: {autoencoder_model_path}")
print(f"✅ Encoder saved at: {encoder_model_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 793 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 28s 45ms/step - loss: 0.0188 
Epoch 2/20


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


250/250 ━━━━━━━━━━━━━━━━━━━━ 23s 13ms/step - loss: 0.0079
Epoch 3/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 0.0056 
Epoch 4/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 0.0042 
Epoch 5/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 0.0041 
Epoch 6/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 0.0036 
Epoch 7/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 0.0038 
Epoch 8/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - loss: 0.0035 
Epoch 9/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 0.0033 
Epoch 10/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - loss: 0.0030 
Epoch 11/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 0.0032 
Epoch 12/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - loss: 0.0027 
Epoch 13/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - loss: 0.0024 
Epoch 14/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 0.0028 
Epoch 15/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 0.0025 
Epoch 16/20
250/250 ━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths
encoder_model_path = '/content/drive/MyDrive/skin_cancer/deep_encoder.keras'
combined_folder = '/content/drive/MyDrive/skin_cancer_images/combined'
csv_output_path = '/content/drive/MyDrive/skin_cancer/deep_extracted_features.csv'

# Load trained encoder
encoder = tf.keras.models.load_model(encoder_model_path)

# Create ImageDataGenerator for loading images
datagen = ImageDataGenerator(rescale=1./255)
data_generator = datagen.flow_from_directory(
    combined_folder,
    target_size=(160, 160),
    batch_size=1,  # Process one image at a time
    class_mode='binary',  # Ensure labels are available
    shuffle=False
)

# Extract labels correctly
labels = data_generator.classes  # Get numeric labels

# Extract features
features = encoder.predict(data_generator)

# 🔥 Flatten features from (batch, 20, 20, 256) → (batch, 102400)
flattened_features = features.reshape(features.shape[0], -1)

# Convert features to DataFrame
df_features = pd.DataFrame(flattened_features)
df_features['label'] = labels  # Add labels column (0 = benign, 1 = malignant)

# Save to CSV
df_features.to_csv(csv_output_path, index=False)
print(f"✅ Features saved at: {csv_output_path}")


Found 793 images belonging to 2 classes.
793/793 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step
✅ Features saved at: /content/drive/MyDrive/skin_cancer/deep_extracted_features.csv


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load extracted features
csv_path = '/content/drive/MyDrive/skin_cancer/deep_extracted_features.csv'
df = pd.read_csv(csv_path)

# Separate features and labels
X = df.drop(columns=['label']).values  # Features
y = df['label'].values  # Labels

# Encode labels (convert class names to numbers)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)  # Converts 'Benign' -> 0, 'Malignant' -> 1

# Split dataset into training and testing (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train & Evaluate SVM
svm_model = SVC(kernel='rbf', C=1.0)
svm_model.fit(X_train, y_train)
svm_preds = svm_model.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_preds)
print(f"🔹 SVM Accuracy: {svm_accuracy:.4f}")

# Train & Evaluate Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_preds = rf_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_preds)
print(f"🔹 Random Forest Accuracy: {rf_accuracy:.4f}")

# Train Neural Network (Simple Feedforward)
nn_model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification
])

nn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the neural network
nn_model.fit(X_train, y_train, epochs=20, batch_size=8, verbose=1, validation_data=(X_test, y_test))

# Evaluate Neural Network
_, nn_accuracy = nn_model.evaluate(X_test, y_test)
print(f"🔹 Neural Network Accuracy: {nn_accuracy:.4f}")


🔹 SVM Accuracy: 0.6415
🔹 Random Forest Accuracy: 0.7107


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - accuracy: 0.5919 - loss: 34.4127 - val_accuracy: 0.5849 - val_loss: 16.0694
Epoch 2/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6242 - loss: 12.4858 - val_accuracy: 0.5723 - val_loss: 6.3749
Epoch 3/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6587 - loss: 8.6109 - val_accuracy: 0.5472 - val_loss: 19.2726
Epoch 4/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6122 - loss: 9.4667 - val_accuracy: 0.6226 - val_loss: 5.7936
Epoch 5/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6815 - loss: 3.7966 - val_accuracy: 0.5409 - val_loss: 3.1303
Epoch 6/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6958 - loss: 2.3529 - val_accuracy: 0.5409 - val_loss: 8.0423
Epoch 7/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6023 - loss: 4.5495 - val_accuracy: 0.5849 - val_loss: 4.9257
Epoch 8/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6953 - loss: 3.5683 - val_accuracy: 0.5094 - val

In [ ]:
print(f"SVM Accuracy: 0.9180")
print(f"Random Forest Accuracy: 0.8427")
print(f"Neural Network Accuracy: 0.9119")

SVM Accuracy: 0.9180
Random Forest Accuracy: 0.8427
Neural Network Accuracy: 0.9119
